In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
book = pd.read_csv('Train_data.csv',engine = 'python')

In [ ]:
testbook = pd.read_csv('Test_data.csv',engine = 'python')

In [ ]:
testbook.info(5)

In [ ]:
#rename attribute
book.rename(columns={'  num_pages':'num_pages'}, inplace = True)
book['dataset'] = 0

In [ ]:
#rename attribute
testbook.rename(columns={'  num_pages':'num_pages'}, inplace = True)
testbook['dataset'] = 1

In [ ]:
#merge for feature engineery
superbook = pd.concat([book,testbook], axis = 0)

In [ ]:
superbook.info()

In [ ]:
#To see any repeat
superbook.duplicated().any()

In [ ]:
superbook.isna().sum()

In [ ]:
#remove null attribute in the publisher columns
superbookC = superbook[superbook['publisher'].notna()]
book = book[book['publisher'].notna()]
testbook = testbook[testbook['publisher'].notna()]

In [ ]:
#changing Three main format type

superbookC['average_rating'] = superbookC['average_rating'].astype('float64')
superbookC['text_reviews_count'] = superbookC['text_reviews_count'].astype('float64')
#book['publication_date'] = pd.to_datetime(book['publication_date'],errors='coerce')

book['average_rating'] = book['average_rating'].astype('float64')
book['text_reviews_count'] = book['text_reviews_count'].astype('float64')


In [ ]:
#check the dataset label
superbookC.loc[superbookC['dataset'] == 1].count()

In [ ]:
#change The date format
years = superbookC['publication_date']
year_type = years.unique()
for year in years:
    #print(type(year))
    superbookC['publication_date'].replace(year, max(list(map(int,str(year).split("/")))), inplace = True)

In [ ]:
#change the date data type
superbookC['publication_date'] = superbookC['publication_date'].astype('float64')

In [ ]:
#remove second translater.
authors = superbookC['authors']
authors_type = authors.unique()
for author in authors:
    superbookC['authors'].replace(author, author.split("/")[0], inplace = True)

In [ ]:
#romeve second translater for orgin dataset
authors = book['authors']
authors_type = authors.unique()
for author in authors:
    book['authors'].replace(author, author.split("/")[0], inplace = True)

In [ ]:
#end of classified rating
#start of turning word in to vector

In [ ]:
superbookC['title'].head(20)
from gensim.models import word2vec

In [ ]:
booktitle = []
title = superbookC['title']
for books in title:
    booktitle.append(books)
    
    
print(booktitle)

In [ ]:
import nltk
import re
nltk.download('punkt')

for i, sentence in enumerate(booktitle):
    word = re.sub('[(#)(()()]',' ',str(booktitle[i]))
    word = nltk.word_tokenize(word)
    booktitle[i] = word
    

In [ ]:
model = word2vec.Word2Vec(booktitle, workers = 1, size = 1, min_count = 1, window = 3, sg = 0)


In [ ]:
Vectorize_list = []
for i in range(len(booktitle)):
    model_list = []
    for word in booktitle[i]:
        model_list.append(model[word])
   
    Vectorize_list.append(sum(model_list)/len(model_list))
    
#end with making title into vector

In [ ]:
#change vector datatype
superbookC['Vectorize Title'] = Vectorize_list
superbookC['Vectorize Title'] = superbookC['Vectorize Title'].astype('float64')

In [ ]:
#language进行分类，英语国家为1，非英语国家为0
def language(language):
    if language in ['eng','en-US','en-GB','en-CA','enm']:
        return 1 #英语国家
    else:
        return 0 #非英语国家
superbookC['language'] = superbookC['language_code'].map(language)




In [ ]:
#author进行计数,即得到该作者写的书的总数
author_type = book['authors'].unique()
def author_bookscount(authors):
    if authors in author_type:
        counts = len(book.loc[book['authors'] == authors])
        return counts
    else:
        return 0
book['author_book_nums'] = book['authors'].map(author_bookscount)

In [ ]:
#处理num_pages的异常值
#替换小于10页的
num_mode = book['num_pages'].mode()
def num_pages(pages):
    if pages <= 10:
        return 10
    else:
        return pages
superbookC['num_pages'] = superbookC['num_pages'].map(num_pages)

#处理ratingcounts=0时,average_rating仍有分数的情况
superbookC.loc[superbookC['ratings_count'] == 0,'average_rating'] = 0

In [ ]:
#make those book who is in series a dummy variable
def if_series(title):
    word = re.sub('[(#)(()()]',' ',str(title))
    word = nltk.word_tokenize(word)
    if_series = 0
    for i in word:
        if(i.isdigit() == True):
            if_series = 1
            break
    return if_series

superbookC['if_series'] = superbookC['title'].map(if_series)

In [ ]:
#superbookC.loc[(superbookC['author_book_nums'] == 1) & (superbookC['if_series'] == 1)]

In [ ]:
#对author进行处理
##筛选出写作数量在五本以上的作家（共506）,同时将五本以下的人分为others,之后进行哑变量处理
def author_group(authors):
    author_group = book.loc[book['author_book_nums']>4,'authors'].unique()
    if authors in author_group:
        return authors
    else:
        return 'others'
book['author_group'] = book['authors'].map(author_group)
book['author_group'].unique()
#五本以上的作者都转化为哑变量
#author_group_dummy= pd.get_dummies(book['author_group'],prefix='author_group')
#superbookC=pd.concat([book,author_group_dummy],axis=1)


In [ ]:
#開始對testdata進行處理
author5 = book['author_group'].unique()
#New column in Dataframe
superbookC['author_group'] = 'others'
superbookC.loc[superbookC['authors'].isin(author5) == True,'author_group'] = superbookC.loc[superbookC['authors'].isin(author5) == True]['authors']

#五本以上的作者都转化为哑变量
author_group_dummy= pd.get_dummies(superbookC['author_group'],prefix='author_group')
superbookC=pd.concat([superbookC,author_group_dummy],axis=1)

In [ ]:
#publisher进行计数,即得到该作者写的书的总数
publisher_type = book['publisher'].unique()
def publisher_bookscount(publisher):
    if publisher in publisher_type:
        counts = len(book.loc[book['publisher'] == publisher])
        return counts
    else:
        return 0
book['publisher_nums'] = book['publisher'].map(publisher_bookscount)

#对publisher进行处理
##筛选出出书数量在五本以上的出版社（共209）,同时将十本以下的出版社分为others,之后进行哑变量处理
def publisher_group(publisher):
    publisher_group = book.loc[book['publisher_nums']>=10,'publisher'].unique()
    if publisher in publisher_group:
        return publisher
    else:
        return 'others'
book['publisher_group'] = book['publisher'].map(publisher_group)


In [ ]:
#開始對testdata進行處理
publisher5 = book['publisher_group'].unique()
#New column in Dataframe
superbookC['publisher_group'] = 'others'
superbookC.loc[superbookC['publisher'].isin(publisher5) == True,'publisher_group'] = superbookC.loc[superbookC['publisher'].isin(publisher5) == True]['publisher']
#十本以上的出版社都转化为哑变量
publisher_group_dummy= pd.get_dummies(superbookC['publisher_group'],prefix='publisher_group')
superbookC=pd.concat([superbookC,publisher_group_dummy],axis=1)

In [ ]:
#split back into test and train
Nbook = superbookC[superbookC['dataset'] == 0]
NTestbook = superbookC[superbookC['dataset'] == 1]

In [ ]:
#end of vectorize the word

In [ ]:
sns.set_context('paper')
plt.figure(figsize=(15,10))
ax = superbookC.groupby('language')['title'].count().plot.bar()
plt.title('Language Code')
plt.xticks(fontsize = 15)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x()-0.3, p.get_height()+100))

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(data=superbookC,x='language',y='average_rating')
plt.xlabel('Languages')
plt.ylabel('Average Rating')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(data=superbookC,y='num_pages',x='language')
plt.xlabel('Language')
plt.ylabel('No. of Pages')
plt.show()

In [ ]:
sns.boxplot(x=superbookC['average_rating'])

In [ ]:
#remove dummy variable
data = Nbook.drop(Nbook.index[Nbook['num_pages'] >= 3000])

In [ ]:
# prepare for the 3trial vairables

In [ ]:
X = data.drop(['publication_date','authors','num_pages','average_rating','language_code', 'isbn','isbn13','title','bookID','text_reviews_count','author_group','publisher_group','dataset','publisher'], axis = 1)
y = data['average_rating']

In [ ]:
XB = Nbook.drop(['publication_date','title','authors','num_pages','average_rating','language_code', 'isbn','isbn13','title','bookID','text_reviews_count','author_group','publisher_group','dataset','publisher'], axis = 1)
yB = Nbook['average_rating']

In [ ]:
UploadX = NTestbook.drop(['publication_date','title','authors','num_pages','average_rating','language_code', 'isbn','isbn13','title','bookID','text_reviews_count','author_group','publisher_group','dataset','publisher'], axis = 1)
Uploady = NTestbook['average_rating']

In [ ]:
# end of prepare for the 3trial variables

In [ ]:
X.info(5)

In [ ]:
# split 80% of the data to the training set and 20% of the data to test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 10)

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(XB, yB, test_size = 0.3, random_state = 10)

In [ ]:
X_train.info()

In [ ]:
#linear Regression
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
predictions = lr.predict(X3_test)

In [ ]:
pred = pd.DataFrame({'Actual': y3_test.tolist(), 'Predicted': predictions.tolist()}).head(25)
pred.head(10)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y3_test, predictions))
print('MSE:', metrics.mean_squared_error(y3_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y3_test, predictions)))
print('score:',  explained_variance_score(y3_test, predictions))

In [ ]:
#Svm
from sklearn import svm
SVM = svm.SVR()
SVM.fit(X_train, y_train)


SVMpredictions = SVM.predict(X3_test)
pred = pd.DataFrame({'Actual': y3_test.tolist(), 'Predicted': SVMpredictions.tolist()}).head(25)
print(pred.head(10))
print('MSE:', metrics.mean_squared_error(y3_test, SVMpredictions))

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

kfold=StratifiedKFold(n_splits=10)

RF_param_grid = {'n_estimators':[1000],
                 'max_depth':[250],
                 'random_state':[1]
                    }

RF=RandomForestRegressor()

modelsRF = GridSearchCV(RF, param_grid=RF_param_grid, cv=5, scoring = 'neg_mean_squared_error',n_jobs = 6)

modelsRF.fit(X_train,y_train)
modelsRF_preds = modelsRF.predict(X3_test)
#forest_model = RandomForestRegressor(random_state=1,n_estimators=160,max_depth = 80,max_features= 'log2')
#forest_model.fit(X_train, y_train)
#forest_preds = forest_model.predict(X_test)
#print(mean_squared_error(y_test, modelsRF_preds))
print(modelsRF.best_params_, mean_squared_error(y3_test, modelsRF_preds))




In [ ]:
#Randomforest Regressor result
modelRF_Result = modelsRF.predict(UploadX)

In [ ]:
testbook['average_rating'] = modelRF_Result
testbook.to_csv('RandomForest_Result.csv',index=False)

In [ ]:
#Trial of Gradient Boost Regressor
from sklearn.ensemble import GradientBoostingRegressor

GBR = GradientBoostingRegressor()

RF_param_grid = {'n_estimators':[1000],
                 'learning_rate':[0.01],
                 'loss':['huber'],
                 'max_depth':[10,100],
                 'random_state':[1]
                    }


GBR_model = GridSearchCV(GBR, param_grid=RF_param_grid, cv=5, scoring = 'neg_mean_squared_error',n_jobs = -1)
GBR_model.fit(X_train, y_train)
GBR_preds = GBR_model.predict(X3_test)
print(GBR_model.best_params_, mean_squared_error(y3_test, GBR_preds))

In [ ]:
#trial of AdaboostRegressor
from sklearn.ensemble import AdaBoostRegressor

AD = AdaBoostRegressor()
print(AD.get_params().keys())
AD_param_grid = {'n_estimators':[1000],
                 'learning_rate':[0.01],
                 'loss':['square'],
                 'random_state':[1]
                    }

Ada_model = GridSearchCV(AD, param_grid=AD_param_grid, cv=5, scoring = 'neg_mean_squared_error',n_jobs = -1)
Ada_model.fit(X_train, y_train)
Ada_preds = Ada_model.predict(X3_test)
print(Ada_model.best_params_, mean_squared_error(y3_test, Ada_preds))

In [ ]:
#trial of Bagging Regressor
from sklearn.ensemble import BaggingRegressor

Bag = BaggingRegressor()
Bag_param_grid = {'n_estimators':[1000],
                 'random_state':[1]}

Bag_model = GridSearchCV(Bag, param_grid=Bag_param_grid, cv=5, scoring = 'neg_mean_squared_error',n_jobs = -1)
Bag_model.fit(X_train, y_train)
Bag_preds = Bag_model.predict(X3_test)
print(Bag_model.best_params_, mean_squared_error(y3_test, Bag_preds))

In [ ]:
#Bagging get the final result
Bag_Result = Bag_model.predict(UploadX)
print(Bag_Result)

In [ ]:
#Bagging output to CSV
testbook['average_rating'] = Bag_Result
testbook.to_csv('Bag_Result.csv',index=False)

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor()


xg_param_grid = {'objective':['reg:tweedie'], 'colsample_bytree': [0.3], 'learning_rate':[0.01],
'max_depth': [200,300], 'alpha':[0.5], 'n_estimators':[1000,1500],'lambda':[5],'seed':[1],'tweedie_variance_power':[1.8,1,3]}

xg_model = GridSearchCV(xg_reg, param_grid=xg_param_grid, cv=5, scoring = 'neg_mean_squared_error',n_jobs = -1)

xg_model.fit(X_train,y_train)
xg_preds = xg_model.predict(X3_test)
print(xg_model.best_params_,mean_squared_error(y3_test, xg_preds))

In [ ]:
xg_result = xg_model.predict(UploadX)
print(xg_result)

In [ ]:
#xg output to csv
testbook['average_rating'] = xg_result
testbook.to_csv('xg_Result.csv',index=False)

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score

ridge = Ridge()
lasso =Lasso()
models = [modelsRF,Bag_model]


params = {
    'ridge__alpha': [0.01, 0.1, 0.5, 1, 3, 5, 7],
    }
sclf = StackingCVRegressor(regressors=models, meta_regressor=ridge, n_jobs = -1)
# 训练回归器
#grid = GridSearchCV(estimator=sclf, param_grid=params, cv=5, refit=True)
sclf.fit(X_train, y_train)

stack_pred = sclf.predict(X3_test)

print(mean_squared_error(y3_test, stack_pred))

In [ ]:
#pytorch test area
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [ ]:
y_train_torch = y_train.values
y_train_torch = np.array(y_train_torch, dtype=np.float32)
y_train_torch = y_train_torch.reshape(-1,1)
y_train_tensor = Variable(torch.from_numpy(y_train_torch))

In [ ]:
X_train_torch = X_train.values
X_train_torch = np.array(X_train_torch, dtype=np.float32)
#X_train_torch = X_train_torch.reshape(-1,1)
X_train_tensor = Variable(torch.from_numpy(X_train_torch))
len(X_train_tensor[1])

In [ ]:
X_test_torch = X3_test.values
X_test_torch = np.array(X_test_torch, dtype=np.float32)
#X_train_torch = X_train_torch.reshape(-1,1)
X_test_tensor = Variable(torch.from_numpy(X_test_torch))


In [ ]:
# Linear Regression with Pytorch

# libraries
import torch      
from torch.autograd import Variable     
import torch.nn as nn 
import warnings
warnings.filterwarnings("ignore")

# create class
class LinearRegression(nn.Module):
    def __init__(self,input_size,output_size):
        # super function. It inherits from nn.Module and we can access everythink in nn.Module
        super(LinearRegression,self).__init__()
        # Linear function.
        self.linear = nn.Linear(input_dim,output_dim)

    def forward(self,x):
        return self.linear(x)

    
    

# define model
input_dim = 722
output_dim = 1
model = LinearRegression(input_dim,output_dim) # input and output size are 1

# MSE
mse = nn.MSELoss()

# Optimization (find parameters that minimize error)
learning_rate = 0.01   # how fast we reach best parameters
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

# train model
loss_list = []
iteration_number = 10000
for iteration in range(iteration_number):
    # optimization
    optimizer.zero_grad() 
    
    # Forward to get output
    results = model(X_train_tensor)
    
    # Calculate Loss
    loss = mse(results, y_train_tensor)
    
    # backward propagation
    loss.backward()
    
    # Updating parameters
    optimizer.step()
    
    # store loss
    loss_list.append(loss.data)
    
    # print loss
    if(iteration % 50 == 0):
        print('epoch {}, loss {}'.format(iteration, loss.data))

plt.plot(range(iteration_number),loss_list)
plt.xlabel("Number of Iterations")
plt.ylabel("Loss")
plt.show()

In [ ]:
predicted = model(X_test_tensor).data.numpy()
print('MSE:', metrics.mean_squared_error(y3_test, predicted))

In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, 
                                                y_train_tensor)

In [ ]:
Batch_size = 100
train_loader = torch.utils.data.DataLoader(train_dataset, Batch_size, 
                                             shuffle=True)

In [ ]:
#nerual network

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import imageio


torch.manual_seed(1)    # reproducible

x = X_train_tensor
y = y_train_tensor             


# 3 layer neural network

class Net(torch.nn.Module):
    def __init__(self,n_input,n_hidden,n_hidden1,n_output):
        super(Net,self).__init__()
        self.hidden=torch.nn.Linear(n_input,n_hidden)
        self.hidden2 = torch.nn.Linear(n_hidden,n_hidden1)
        self.predict=torch.nn.Linear(n_hidden,n_output)

    
    def forward(self,X):
        X=F.relu(self.hidden(X))
        X=F.relu(self.hidden2(X))
        
        #我猜這裡已經去掉了activation layer relu
        X=self.predict(X)
        return X
    
#seting parameter
net=Net(n_input=722,n_hidden=500,n_hidden1=500,n_output=1)
loss_func=torch.nn.MSELoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.01)
loss_List=[]
epoch = 500

for t in range(epoch):
    for  i, (data, labels) in enumerate(train_loader):
        prediction=net(data)
        loss = mse(prediction, labels)
        loss_List.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(epoch % 50 == 0):
            print('epoch {}, loss {}'.format(t, loss.data))



In [ ]:
predicted = net(X_test_tensor).data.numpy()
print('MSE:', metrics.mean_squared_error(y3_test, predicted))